In [150]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
import random
import torch
import torch.nn as nn

import torch.optim as optim

from datetime import datetime, timedelta
from torch.autograd import Variable
import torch.nn.functional as F

In [151]:
def DataClean(f1,f2):
	with open(f1) as f:
		data = json.load(f)
	for i in data.values():
		for j in i:
			del j["place_vector"]
			del j["city_vector"]
	json.dump(data,open(f2,'w'),ensure_ascii=False)

In [152]:
# for i in range(3):
# 	DataClean('classified_data_'+str(i)+'.json','data_'+str(i)+'.json')

In [153]:
with open('data_0.json') as f:
	d0 = json.load(f)
with open('data_1.json') as f:
	d1 = json.load(f)
with open('data_2.json') as f:
	d2 = json.load(f)

In [154]:
input_size = 0
for city in d2.values():
	if input_size != 0:
		break
	for p in city:
		if type(p['patient_vector']) == list:
			input_size = len(p['patient_vector'])
			break

In [155]:
input_size

300

In [156]:
total = 0
useful = 0
for city in d0.values():
	total += len(city)
	if len(city)>70:
		print(city[0]['true_city'])
	for p in city:
		if type(p['patient_vector']) == list:
			useful += 1

重庆
泉州
南京


In [157]:
total = 0
useful = 0
for city in d1.values():
	total += len(city)
	if len(city)>70:
		print(city[0]['true_city'])
	for p in city:
		if type(p['patient_vector']) == list:
			useful += 1

兰州
重庆
泉州
南京


In [158]:
# d1['南京']

In [159]:
total = 0
useful = 0
for city in d2.values():
	total += len(city)
	if len(city)>30:
		print(city[0]['true_city'])
	for p in city:
		if type(p['patient_vector']) == list:
			useful += 1


兰州
泉州
南京


In [160]:
tset = set()
for p in d0['泉州']:
	tset.add(p['time'])
tset

{'2021:08:30',
 '2021:09:08',
 '2021:09:09',
 '2021:09:10',
 '2021:09:11',
 '2021:09:12',
 '2021:09:16',
 '2022:03:08',
 '2022:03:09',
 '2022:03:10',
 '2022:03:11',
 '2022:03:12',
 '2022:03:13',
 '2022:03:14',
 '2022:03:15',
 '2022:03:16',
 '2022:03:17',
 '2022:03:18',
 '2022:03:19',
 '2022:03:20',
 '2022:03:21',
 '2022:03:22',
 '2022:03:23'}

In [161]:

datas = {}
for city_string in ('兰州','重庆','南京','泉州'):
	city = d0[city_string]
	start_date = datetime.strptime(city[0]['time'],'%Y:%m:%d')
	end_date = datetime.strptime(city[-1]['time'],'%Y:%m:%d')
	# start_date = datetime.strptime('2022:03:08','%Y:%m:%d')
	# end_date = datetime.strptime('2022:03:23','%Y:%m:%d')
	CNT = []
	VECTOR = []
	while start_date < end_date:
		start_date += timedelta(days=1)
		cnt = 0
		vector = []
		for p in city:
			if p['time']==(start_date).strftime('%Y:%m:%d'):
				cnt+=1
				vector.append(p['patient_vector'])
		if any((type(v)==list for v in vector)):
			vector = np.array([v for v in vector if type(v)==list])
			vector = np.mean(vector,axis=0)
		else:
			vector = np.zeros((input_size,))
		CNT.append(cnt)
		VECTOR.append(vector)
	CNT = np.array(CNT)
	VECTOR = np.array(VECTOR)
	temp = np.hstack((CNT.reshape(-1,1),VECTOR))

	datas[city_string] = torch.tensor(temp.reshape(-1,1,input_size+1)).float()
input_size += 1

In [162]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [163]:
len(datas)

4

In [164]:
class Net(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(Net, self).__init__()
        self.gru = nn.GRU(input_size=input_size, \
            hidden_size=hidden_size)
        self.linear = nn.Linear(hidden_size,output_size) 
        self.relu = nn.ReLU()

    def forward(self, x):
        output, h_n = self.gru(x)
        # output = self.relu(output[-1])
        output = self.linear(output)
        output = self.relu(output[-1])
        return output

net = Net(input_size,16,1)
# net.to(device)

In [165]:

# create your optimizer
optimizer = optim.Adam(net.parameters(), lr=0.01)
criterion = nn.MSELoss()


In [166]:
def data_iter(data,seq_len):
	indexs = np.arange(len(data)-seq_len)
	random.shuffle(indexs)
	for i in indexs:
		# print(data[i:i+7].shape)
		yield data[i:i+seq_len],data[i+seq_len][0][0]

In [167]:
epoches = 50
seq_len = 7
for epoch in tqdm(range(epoches)):  # loop over the dataset multiple times

    running_loss = 0.0
    for city in datas.values():
        for i,data in enumerate(data_iter(city,seq_len)):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            # inputs, labels = inputs.to(device), labels.view(1,1).to(device)
            inputs, labels = inputs, labels.view(1,1)
            # raise Exception(inputs.size)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            # if labels != torch.tensor(0.,):
            #     raise Exception(labels.size(),labels)
            # else:
            #     raise Exception(inputs,labels)
            # if not i: print(outputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

    if epoch%5 == 4:
        print(epoch+1,running_loss)

    running_loss = 0.0

print('Finished Training')

 10%|█         | 5/50 [00:10<01:33,  2.07s/it]

5 3889.5673598822905


 20%|██        | 10/50 [00:19<01:16,  1.92s/it]

10 3577.5669859847403


 30%|███       | 15/50 [00:29<01:06,  1.91s/it]

...

40 2521.448900565505


 90%|█████████ | 45/50 [01:25<00:09,  1.98s/it]

45 2022.545015200274


100%|██████████| 50/50 [01:37<00:00,  1.95s/it]

50 1633.5522323632904
Finished Training


In [168]:
for city_string in ('兰州','重庆','南京','泉州'):

	inputs = datas[city_string][-8:-1]
	outputs = net(inputs)
	print(city_string)
	print(outputs)
	print(datas[city_string][-1][0][0])

兰州
tensor([[0.]], grad_fn=<ReluBackward0>)
tensor(1.)
重庆
tensor([[0.]], grad_fn=<ReluBackward0>)
tensor(1.)
南京
tensor([[0.]], grad_fn=<ReluBackward0>)
tensor(1.)
泉州
tensor([[0.9251]], grad_fn=<ReluBackward0>)
tensor(1.)


In [169]:
# for i in net.gru.parameters():
# 	print(i)